# Inflating a normal cylinder

In [ ]:
from typing import Literal
from pathlib import Path
from mpi4py import MPI
import dolfinx
import logging
from dolfinx import log
import ufl
import numpy as np
from scipy.integrate import solve_ivp
import adios4dolfinx
import pulse
import cardiac_geometries
import cardiac_geometries.geometry
import pyvista

In [ ]:
comm = MPI.COMM_WORLD
target_pressure = 10_000  # Pa
char_length = 0.005
r_inner = 0.02
r_outer = 0.03
height = 0.05
outdir = Path("results_normal_cylinder")
geodir = outdir / "geometry"
fiber_angle = 0.0

In [ ]:
if not geodir.exists():
    cardiac_geometries.mesh.cylinder(
        outdir=geodir,
        create_fibers=True,
        fiber_space="DG_1",
        r_inner=r_inner,
        r_outer=r_outer,
        height=height,
        char_length=char_length,
        comm=comm,
        fiber_angle_epi=-fiber_angle,
        fiber_angle_endo=fiber_angle,
    )

In [ ]:
# If the folder already exist, then we just load the geometry
geo = cardiac_geometries.geometry.Geometry.from_folder(
    comm=comm,
    folder=geodir,
)

In [ ]:
vtk_mesh = dolfinx.plot.vtk_mesh(geo.mesh, geo.mesh.topology.dim)
grid = pyvista.UnstructuredGrid(*vtk_mesh)
plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    plotter.screenshot(outdir / "cylinder_mesh.png")

In [ ]:
assert geo.f0 is not None
topology, cell_types, geometry = dolfinx.plot.vtk_mesh(geo.f0.function_space)
topology, cell_types, geometry = dolfinx.plot.vtk_mesh(geo.f0.function_space)
values = np.zeros((geometry.shape[0], 3), dtype=np.float64)
values[:, : len(geo.f0)] = geo.f0.x.array.real.reshape((geometry.shape[0], len(geo.f0)))
function_grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
function_grid["u"] = values
glyphs = function_grid.glyph(orient="u", factor=char_length)
grid = pyvista.UnstructuredGrid(*vtk_mesh)
plotter = pyvista.Plotter()
plotter.add_mesh(grid, style="wireframe", color="r")
plotter.add_mesh(glyphs)
plotter.view_xy()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    plotter.screenshot(outdir / "cylinder_fiber.png")

Next we transform the geometry to a `HeartGeometry` object

In [ ]:
geometry = pulse.HeartGeometry.from_cardiac_geometries(
    geo, metadata={"quadrature_degree": 6}
)

In [ ]:
# Longitudinal direction
l0 = dolfinx.fem.Constant(geometry.mesh, (0.0, 0.0, 1.0))
# Next we create the material object, and we will use the transversely isotropic version of the {py:class}`Holzapfel Ogden model <pulse.holzapfelogden.HolzapfelOgden>`

In [ ]:
# material_params = pulse.HolzapfelOgden.transversely_isotropic_parameters()
# material = pulse.HolzapfelOgden(f0=geo.f0, s0=geo.s0, **material_params)  # type: ignore
mu = pulse.Variable(dolfinx.fem.Constant(geometry.mesh, 10.0), "kPa")
material = pulse.NeoHookean(mu=mu)

In [ ]:
active_model = pulse.active_model.Passive()

In [ ]:
comp_model = pulse.compressibility.Compressible2()

and assembles the `CardiacModel`

In [ ]:
model = pulse.CardiacModel(
    material=material,
    active=active_model,
    compressibility=comp_model,
)

In [ ]:
traction = pulse.Variable(
    dolfinx.fem.Constant(geometry.mesh, dolfinx.default_scalar_type(0.0)),
    "Pa",
)
robin_value_epi = pulse.Variable(
    dolfinx.fem.Constant(geometry.mesh, dolfinx.default_scalar_type(2e6)),
    "Pa / m",
)
robin_value_base = pulse.Variable(
    dolfinx.fem.Constant(geometry.mesh, dolfinx.default_scalar_type(8e6)),
    "Pa / m",
)
neumann: tuple[pulse.NeumannBC, ...] = (
    pulse.NeumannBC(traction=traction, marker=geometry.markers["INSIDE"][0]),
)
robin: tuple[pulse.RobinBC, ...] = (
    pulse.RobinBC(value=robin_value_base, marker=geometry.markers["TOP"][0]),
    pulse.RobinBC(value=robin_value_base, marker=geometry.markers["BOTTOM"][0]),
    pulse.RobinBC(value=robin_value_epi, marker=geometry.markers["OUTSIDE"][0]),
)
parameters = pulse.problem.StaticProblem.default_parameters()
parameters["mesh_unit"] = "m"
# Set null space options for MUMPS solver to handle rigid body motions
# parameters["petsc_options"]["mat_mumps_icntl_24"] = 1  # Zero pivot detection
# parameters["petsc_options"]["mat_mumps_icntl_25"] = 0  # Which nullspace to extract
# parameters["petsc_options"]["mat_mumps_icntl_4"] = 1  # Verbosity
# parameters["petsc_options"]["mat_mumps_icntl_2"] = 1  # std out blaaah
# parameters["petsc_options"]["mat_mumps_cntl_3"] = 1e-6  # Threshold factor

In [ ]:
# Next we set up the problem.
# bcs = pulse.BoundaryConditions(robin=robin, neumann=neumann, dirichlet=(dirichlet_bc,))
bcs = pulse.BoundaryConditions(robin=robin, neumann=neumann)
problem = pulse.problem.StaticProblem(
    model=model,
    geometry=geometry,
    bcs=bcs,
    parameters=parameters,
)

In [ ]:
W = dolfinx.fem.functionspace(geometry.mesh, ("DG", 1))
W_tensor = dolfinx.fem.functionspace(geometry.mesh, ("DG", 1, (3, 3)))
I = ufl.Identity(3)
F = ufl.variable(ufl.grad(problem.u) + I)
C = F.T * F
E = 0.5 * (C - I)
T = material.sigma(F)
S = material.S(ufl.variable(C))

In [ ]:
# Fibers in current configuration
f = (F * geo.f0) / ufl.sqrt(ufl.inner(F * geo.f0, F * geo.f0))
l = (F * l0) / ufl.sqrt(ufl.inner(F * l0, F * l0))

In [ ]:
fiber_stress = dolfinx.fem.Function(W, name="fiber_stress")
fiber_stress_expr = dolfinx.fem.Expression(
    ufl.inner(T * f, f),
    W.element.interpolation_points,
)
longitudinal_stress = dolfinx.fem.Function(W, name="longitudinal_stress")
longitudinal_stress_expr = dolfinx.fem.Expression(
    ufl.inner(T * l, l),
    W.element.interpolation_points,
)
fiber_strain = dolfinx.fem.Function(W, name="fiber_strain")
fiber_strain_expr = dolfinx.fem.Expression(
    ufl.inner(E * geo.f0, geo.f0),
    W.element.interpolation_points,
)
longitudinal_strain = dolfinx.fem.Function(W, name="longitudinal_strain")
longitudinal_strain_expr = dolfinx.fem.Expression(
    ufl.inner(E * l0, l0),
    W.element.interpolation_points,
)

Now we can solve the problem

In [ ]:
log.set_log_level(log.LogLevel.INFO)
problem.solve()

In [ ]:
vtx = dolfinx.io.VTXWriter(
    geometry.mesh.comm,
    f"{outdir}/displacement.bp",
    [problem.u],
    engine="BP4",
)
vtx.write(0.0)
vtx_stress = dolfinx.io.VTXWriter(
    geometry.mesh.comm,
    f"{outdir}/stress.bp",
    [fiber_stress, longitudinal_stress, fiber_strain, longitudinal_strain],
    engine="BP4",
)
vtx_stress.write(0.0)

In [ ]:
for p in np.linspace(0, target_pressure, 5)[1:]:
    print(f"Solving for pressure {p} Pa")
    traction.assign(p)

    problem.solve()
    fiber_strain.interpolate(fiber_strain_expr)
    fiber_stress.interpolate(fiber_stress_expr)
    longitudinal_strain.interpolate(longitudinal_strain_expr)
    longitudinal_stress.interpolate(longitudinal_stress_expr)

    vtx.write(p)
    vtx_stress.write(p)

## Visualization of results

In [ ]:
u_topology, u_cell_types, u_geometry = dolfinx.plot.vtk_mesh(W)
u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
u_grid.point_data["fiber_strain"] = fiber_strain.x.array
u_grid.point_data["fiber_stress"] = fiber_stress.x.array
u_grid.point_data["longitudinal_strain"] = longitudinal_strain.x.array
u_grid.point_data["longitudinal_stress"] = longitudinal_stress.x.array

## Fiber strain

In [ ]:
u_grid.set_active_scalars("fiber_strain")
u_plotter = pyvista.Plotter(off_screen=True)
u_plotter.add_mesh(u_grid)
# u_plotter.add_mesh_clip_plane(u_grid, origin=(96.5, 116.5, 171.5), normal="-z")
u_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    u_plotter.show()
else:
    u_plotter.screenshot(outdir / "cylinder_fiber_strain.png")

Clip in the $z-direction$

In [ ]:
u_plotter = pyvista.Plotter(off_screen=True)
u_plotter.add_mesh_clip_plane(u_grid, normal="-z")
u_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    u_plotter.show()
else:
    u_plotter.screenshot(outdir / "cylinder_fiber_strain.png")

Clip in the $x$-direction

In [ ]:
u_plotter = pyvista.Plotter(off_screen=True)
u_plotter.add_mesh_clip_plane(u_grid, normal="x")
u_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    u_plotter.show()
else:
    u_plotter.screenshot(outdir / "cylinder_fiber_strain.png")

## Fiber stress

In [ ]:
u_grid.set_active_scalars("fiber_stress")
u_plotter = pyvista.Plotter(off_screen=True)
u_plotter.add_mesh(u_grid)
# u_plotter.add_mesh_clip_plane(u_grid, origin=(96.5, 116.5, 171.5), normal="-z")
u_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    u_plotter.show()
else:
    u_plotter.screenshot(outdir / "cylinder_fiber_stress.png")

Clip in the $z-direction$

In [ ]:
u_plotter = pyvista.Plotter(off_screen=True)
u_plotter.add_mesh_clip_plane(u_grid, normal="-z")
u_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    u_plotter.show()
else:
    u_plotter.screenshot(outdir / "cylinder_fiber_stress.png")

Clip in the $x$-direction

In [ ]:
u_plotter = pyvista.Plotter(off_screen=True)
u_plotter.add_mesh_clip_plane(u_grid, normal="x")
u_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    u_plotter.show()
else:
    u_plotter.screenshot(outdir / "cylinder_fiber_stress.png")

## Longitudinal strain

In [ ]:
u_grid.set_active_scalars("longitudinal_strain")
u_plotter = pyvista.Plotter(off_screen=True)
u_plotter.add_mesh(u_grid)
# u_plotter.add_mesh_clip_plane(u_grid, origin=(96.5, 116.5, 171.5), normal="-z")
u_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    u_plotter.show()
else:
    u_plotter.screenshot(outdir / "cylinder_longitudinal_strain.png")

Clip in the $z-direction$

In [ ]:
u_plotter = pyvista.Plotter(off_screen=True)
u_plotter.add_mesh_clip_plane(u_grid, normal="-z")
u_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    u_plotter.show()
else:
    u_plotter.screenshot(outdir / "cylinder_longitudinal_strain.png")

Clip in the $x$-direction

In [ ]:
u_plotter = pyvista.Plotter(off_screen=True)
u_plotter.add_mesh_clip_plane(u_grid, normal="x")
u_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    u_plotter.show()
else:
    u_plotter.screenshot(outdir / "cylinder_longitudinal_strain.png")

## Longitudinal stress

In [ ]:
u_grid.set_active_scalars("longitudinal_stress")
u_plotter = pyvista.Plotter(off_screen=True)
u_plotter.add_mesh(u_grid)
# u_plotter.add_mesh_clip_plane(u_grid, origin=(96.5, 116.5, 171.5), normal="-z")
u_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    u_plotter.show()
else:
    u_plotter.screenshot(outdir / "cylinder_longitudinal_stress.png")

Clip in the $z-direction$

In [ ]:
u_plotter = pyvista.Plotter(off_screen=True)
u_plotter.add_mesh_clip_plane(u_grid, normal="-z")
u_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    u_plotter.show()
else:
    u_plotter.screenshot(outdir / "cylinder_longitudinal_stress.png")

Clip in the $x$-direction

In [ ]:
u_plotter = pyvista.Plotter(off_screen=True)
u_plotter.add_mesh_clip_plane(u_grid, normal="x")
u_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    u_plotter.show()
else:
    u_plotter.screenshot(outdir / "cylinder_longitudinal_stress.png")